Use the extracted features to predict in-hospital mortality, treating each timestep as an example. 

Evaluate on the test set using AUROC. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('../data/rl_train_set_scaled.csv')
df_val =  pd.read_csv('../data/rl_val_set_scaled.csv')
df_test = pd.read_csv('../data/rl_test_set_scaled.csv')

In [3]:
df_train

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,output_total,output_4hourly,cumulated_balance,SOFA,SIRS,vaso_input,iv_input,vaso_input_NEW,iv_input_NEW,terminal_reward
0,0.000000,3,7245486000,0.0,0.412568,0.0,0.0,0,0,1,...,0.787595,0.717533,0.322651,0.166667,0.25,0,2,0,1,0
1,0.222560,3,7245500400,0.0,0.412568,0.0,0.0,0,0,1,...,0.789357,0.687860,0.322334,0.291667,0.50,0,2,0,1,0
2,0.356608,3,7245514800,0.0,0.412568,0.0,0.0,0,0,1,...,0.790678,0.665636,0.322103,0.208333,0.50,0,2,0,1,0
3,0.452837,3,7245529200,0.0,0.412568,0.0,0.0,0,0,1,...,0.792248,0.681912,0.321812,0.208333,0.25,0,2,0,1,0
4,0.527957,3,7245543600,0.0,0.412568,0.0,0.0,0,0,1,...,0.794667,0.720697,0.321319,0.208333,0.50,0,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195298,0.727252,99995,4612888440,0.0,0.072993,0.0,0.0,0,0,0,...,0.716156,0.729530,0.327221,0.000000,0.50,0,0,0,0,0
195299,0.763137,99995,4612902840,0.0,0.072993,0.0,0.0,0,0,0,...,0.725480,0.742455,0.326517,0.000000,0.50,0,0,0,0,0
195300,0.795631,99995,4612917240,0.0,0.072993,0.0,0.0,0,0,0,...,0.725480,0.000000,0.326517,0.291667,0.50,0,0,0,0,0
195301,0.825321,99995,4612931640,0.0,0.072993,0.0,0.0,0,0,0,...,0.730506,0.699627,0.326094,0.000000,0.50,0,0,0,0,0


In [4]:
binary_fields = ['gender','mechvent','re_admission']
norm_fields= ['age','Weight_kg','GCS','HR','SysBP','MeanBP','DiaBP','RR','Temp_C','FiO2_1',
    'Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium',
    'Hb','WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2',
    'Arterial_BE','HCO3','Arterial_lactate','SOFA','SIRS','Shock_Index',
    'PaO2_FiO2','cumulated_balance', 'elixhauser', 'Albumin', u'CO2_mEqL', 'Ionised_Ca']
log_fields = ['max_dose_vaso','SpO2','BUN','Creatinine','SGOT','SGPT','Total_bili','INR',
              'input_total','input_4hourly','output_total','output_4hourly', 'bloc']

In [5]:
feature_cols = binary_fields + norm_fields + log_fields

In [6]:
len(feature_cols)

52

In [7]:
df_train[df_train.columns.difference(feature_cols)]

,charttime,delay_end_of_record_and_discharge_or_death,died_in_hosp,died_within_48h_of_out_time,icustayid,iv_input,iv_input_NEW,median_dose_vaso,mortality_90d,terminal_reward,vaso_input,vaso_input_NEW
0,7245486000,326.666667,0,0,3,2,1,0.0,1,0,0,0
1,7245500400,326.666667,0,0,3,2,1,0.0,1,0,0,0
2,7245514800,326.666667,0,0,3,2,1,0.0,1,0,0,0
3,7245529200,326.666667,0,0,3,2,1,0.0,1,0,0,0
4,7245543600,326.666667,0,0,3,2,1,0.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
195298,4612888440,24.000000,0,0,99995,0,0,0.0,0,0,0,0
195299,4612902840,24.000000,0,0,99995,0,0,0.0,0,0,0,0
195300,4612917240,24.000000,0,0,99995,0,0,0.0,0,0,0,0
195301,4612931640,24.000000,0,0,99995,0,0,0.0,0,0,0,0


In [8]:
X_tr = df_train[feature_cols].values
X_va = df_val[feature_cols].values
X_te = df_test[feature_cols].values
y_tr = df_train['died_in_hosp'].values
y_va = df_val['died_in_hosp'].values
y_te = df_test['died_in_hosp'].values

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [11]:
clf = LogisticRegression(random_state=0)

In [12]:
clf.fit(X_tr, y_tr)

/data4/tangsp/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
metrics.roc_auc_score(y_va, clf.predict_proba(X_va)[:,1])

0.805472050085174